In [1]:
import os
os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.metrics import AUC
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.models import Sequential

In [3]:
df_train = pd.read_csv('data/feature_engineered/null_cat/train_scaled_for_not_gbdt.csv')
df_test = pd.read_csv('data/feature_engineered/null_cat/test_scaled_for_not_gbdt.csv')

In [4]:
feature = [
    'Age', 
    'DurationOfPitch', 
    'NumberOfPersonVisiting',
    'NumberOfFollowups', 
    'NumberOfTrips', 
    'MonthlyIncome', 
    #'ProdTaken',
    'Motivation', 
    'EconomicPower', 
    'TripEasier', 
    'SalesPerformance',
    'LivingCost', 
    'EconomicStability', 
    'NumberOfTrips_log', 
    'TravelCost',
    'EconomicSegment', 
    'PackageMatch', 
    'Monetary', 
    #'ContractRate_FM',
    #'ContractRate_G1',
    #'ContractRate_G2', 
    #'ContractRate_G3',
    #'ContractRate_G4',
    #'ContractRate_G5', 
    #'ContractRate_G6',
    'TypeofContact_No',
    'TypeofContact_Self Enquiry',
    'CityTier_2',
    'CityTier_3',
    'Occupation_Salaried',
    'Occupation_Small Business',
    'Gender_male', 
    'ProductPitched_Deluxe', 
    'ProductPitched_King',
    'ProductPitched_Standard',
    'ProductPitched_Super Deluxe',
    'PreferredPropertyStar_4',
    'PreferredPropertyStar_5', 
    'Passport_1',
    'PitchSatisfactionScore_2', 
    'PitchSatisfactionScore_3',
    'PitchSatisfactionScore_4',
    'PitchSatisfactionScore_5',
    'Designation_Executive',
    'Designation_Manager',
    'Designation_Senior Manager', 
    'Designation_VP',
    'Marry_Married',
    'Marry_Single', 
    'Car_No Car', 
    'Child_1_child', 
    'Child_2_child',
    'Child_3_child',
    'AgeGroup_20s',
    'AgeGroup_30s',
    'AgeGroup_40s',
    'AgeGroup_50s', 
    'AgeGroup_60s', 
    'TypeofContactNULL_1',
    'Child01_1',
    'IsFamily_1',
    'FreaqencySeg_1',
    'FreaqencySeg_2',
    'MonetarySeg_2',
    'MonetarySeg_3', 
    'MonetarySeg_4', 
    'AgeNull', 
    'DurationOfPitchNull',
    'NumberOfTripsNull', 
    'MonthlyIncomeNull'
]

X = df_train[feature]
y = df_train['ProdTaken']

In [10]:
scores = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for tr_idx, va_idx in skf.split(X, y):
    tr_x, va_x = X.iloc[tr_idx], X.iloc[va_idx]
    tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
    
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])
    
    batch_size = 128
    epochs = 10
    history = model.fit(tr_x, tr_y,
                        batch_size=batch_size, epochs=epochs,
                        verbose=1, validation_data=(va_x, va_y))
    
    pred = model.predict(va_x)
    score = roc_auc_score(va_y, pred)
    scores.append(score)

print('AUC : ', scores)

Epoch 1/10
22/22 [==============================] - 1s 21ms/step - loss: 0.7490 - auc_15: 0.6770 - val_loss: 0.6400 - val_auc_15: 0.7984
Epoch 2/10
22/22 [==============================] - 0s 5ms/step - loss: 0.6479 - auc_15: 0.7465 - val_loss: 0.5851 - val_auc_15: 0.8258
Epoch 3/10
22/22 [==============================] - 0s 6ms/step - loss: 0.5876 - auc_15: 0.7799 - val_loss: 0.5408 - val_auc_15: 0.8351
Epoch 4/10
22/22 [==============================] - 0s 6ms/step - loss: 0.5517 - auc_15: 0.7874 - val_loss: 0.5325 - val_auc_15: 0.8315
Epoch 5/10
22/22 [==============================] - 0s 5ms/step - loss: 0.5251 - auc_15: 0.7988 - val_loss: 0.4936 - val_auc_15: 0.8336
Epoch 6/10
22/22 [==============================] - 0s 6ms/step - loss: 0.4950 - auc_15: 0.8153 - val_loss: 0.4813 - val_auc_15: 0.8280
Epoch 7/10
22/22 [==============================] - 0s 6ms/step - loss: 0.4801 - auc_15: 0.8164 - val_loss: 0.4709 - val_auc_15: 0.8257
Epoch 8/10
22/22 [=============================